### Il est explicité ici l'utilisation des ensembles de codes écrits pour l'analyse de sensibilité sur des champs stochastiques et variables aléatoires.

### Il est d'abord conseillé d'installer l'environnement virtuel dont la définition se trouvent dans le fichier yaml

Nous allons faire ici l'analyse de sensiblité sur une poutre en flexion représentée par 100 éléments finis, et ou le module young et le diamètre de chaque élément est déterminé par un processus gaussien en une dimension. La position de la force, sa norme, tout comme la densité du matérieau sont déterminés par des lois nomales gaussiennes.
Dans la logique d'écriture de ces codes, il faut avoir un à priori sur les processus gaussiens, la loi qu'ils suivent, tout comme sur les paramètres des lois gaussiennes. Ensuite, il faut avoir une fonction python qui prend en entrée ces champs, et qui renvoie un ensemble connu de resultas. 


In [1]:
### Essai pour voir si les packages nécessaires sont installés
try:
    import anastruct, openturns, numba, joblib
except:
    import os
    if os.sys.platform == 'linux' :
        file_path = 'sensitivityEnv.yml'
        os.system('conda env create -f'+file_path)
        # to have the right modules installed
        print('now activate the environment and restart jupyter with other kernel')
    else :
        print('Do it alone')

In [2]:
import sys
sys.path.insert(0, '../')
# Voici les deux scripts destinés à gérer l'analyse de sensibilité sur les champs stochastiques
import spsa as spsa
# Classes utilitaires
import numpy                        as np
import openturns                    as ot
import matplotlib.pyplot            as plt
from   importlib                import reload 
# on importe aussi les fonctions à étudier
import beamExample as PBE

len vertices is: 101


D'abord, nous définisson l'ensemble de nos variables d'entrée, tout comme les variables de sortie :
- L'on va définir un par un tout les processus et variables aléatoires utilisées dans notre model. Bien sûr cela implique d'avoir un à-priori sur le comportement de ces différentes lois probabilistes. 
- Néanmoins, comme nous en sommes à des codes d'essai, il serait assez trivial de rajouter la prossiblité de récuperer l'approximation d'un champ inconnu avec l'approcimation de Karhunen - Loeve, en présence d'un grand nombre de mesures.
- La définition des éléments sur lesquels est construit le champ stochastique est un peu différent de la manière interne à openturns. En effet, s'étant placés directement dans un cadre de poutre en 'éléments finis, ou l'on a N+1 noeuds (N étant le nombre de poutres), il faut savoir si N est le nombre de mailles ou le nombre de noeuds. Dans le choix a été fait ici de définir la taille du maillage (grid_shape) de la manière suivante :
> grid_shape = [[position_X0, longeur_totaleX, nombre_mailles], [position_Y0, longeur_totaleY, nombre_mailles], ..]
 

In [14]:
# process governing the young modulus for each element (MPa)
process_E = spsa.StochasticProcessConstructor(dimension=1,
                                        grid_shape=[[0,1000,99],],
                                        covariance_model={'Model':'MaternModel',
                                                        'amplitude':[50000],
                                                        'scale':[300],
                                                        'nu':13/3},
                                        trend_arguments=['x'],trend_function=210000)
process_E.setName('E_')


###########################################################################################################
# process governing the diameter for each element (mm) 
# other constructor shown 
process_D = spsa.StochasticProcessConstructor()
process_D.setDimension(1)
process_D.setGrid([[0,1000,99],])
#setCovarianceModel also accepts any of the models defined in openturns, already initialised
#with the right parameters, not only dictionaries
process_D.setCovarianceModel(covarianceModelDict = {'Model':'MaternModel',
                                                    'amplitude':[.3],
                                                    'scale':[250],
                                                    'nu':7.4/3})
process_D.setTrend(['X'], 10)
process_D.setName('D_')

###########################################################################################################
# random variable for the position of the force   (mm) 
middle       = 500
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ot.Normal(middle, sigma_f)
RV_Fpos.setName(namePos)

###########################################################################################################
# random variable for the norm of the force    (N)
muForce       = 100
# we go from sigma = 15 to sigma = 1.5, as the influence is too important
sigma_Fnor    = 5.5
nameNor       = 'FN'
RV_Fnorm  = ot.Normal(muForce, sigma_Fnor)
RV_Fnorm.setName(nameNor)

In [15]:
process_E.getRealization()

class=Field name=Unnamed description=[v0,y0] implementation=class=FieldImplementation name=Unnamed mesh=class=Mesh name=1D_Grid dimension=1 vertices=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=100 dimension=1 data=[[0],[10.101],[20.202],[30.303],[40.404],[50.5051],[60.6061],[70.7071],[80.8081],[90.9091],[101.01],[111.111],[121.212],[131.313],[141.414],[151.515],[161.616],[171.717],[181.818],[191.919],[202.02],[212.121],[222.222],[232.323],[242.424],[252.525],[262.626],[272.727],[282.828],[292.929],[303.03],[313.131],[323.232],[333.333],[343.434],[353.535],[363.636],[373.737],[383.838],[393.939],[404.04],[414.141],[424.242],[434.343],[444.444],[454.545],[464.646],[474.747],[484.848],[494.949],[505.051],[515.152],[525.253],[535.354],[545.455],[555.556],[565.657],[575.758],[585.859],[595.96],[606.061],[616.162],[626.263],[636.364],[646.465],[656.566],[666.667],[676.768],[686.869],[696.97],[707.071],[717.172],[727.273],[737.374],[747.475],[757.576],[767.677],[777.778],[787.879],[797.98],[808.081],[818.182],[828.283],[838.384],[848.485],[858.586],[868.687],[878.788],[888.889],[898.99],[909.091],[919.192],[929.293],[939.394],[949.495],[959.596],[969.697],[979.798],[989.899],[1000]] simplices=[[0,1],[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9,10],[10,11],[11,12],[12,13],[13,14],[14,15],[15,16],[16,17],[17,18],[18,19],[19,20],[20,21],[21,22],[22,23],[23,24],[24,25],[25,26],[26,27],[27,28],[28,29],[29,30],[30,31],[31,32],[32,33],[33,34],[34,35],[35,36],[36,37],[37,38],[38,39],[39,40],[40,41],[41,42],[42,43],[43,44],[44,45],[45,46],[46,47],[47,48],[48,49],[49,50],[50,51],[51,52],[52,53],[53,54],[54,55],[55,56],[56,57],[57,58],[58,59],[59,60],[60,61],[61,62],[62,63],[63,64],[64,65],[65,66],[66,67],[67,68],[68,69],[69,70],[70,71],[71,72],[72,73],[73,74],[74,75],[75,76],[76,77],[77,78],[78,79],[79,80],[80,81],[81,82],[82,83],[83,84],[84,85],[85,86],[86,87],[87,88],[88,89],[89,90],[90,91],[91,92],[92,93],[93,94],[94,95],[95,96],[96,97],[97,98],[98,99]] values=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=100 dimension=1 description=[y0] data=[[177479],[176009],[174582],[173203],[171870],[170581],[169334],[168126],[166958],[165833],[164754],[163721],[162733],[161786],[160877],[160005],[159168],[158370],[157612],[156896],[156222],[155597],[155035],[154544],[154136],[153819],[153602],[153491],[153496],[153625],[153883],[154271],[154790],[155440],[156214],[157106],[158110],[159223],[160441],[161759],[163171],[164668],[166242],[167893],[169621],[171431],[173322],[175294],[177343],[179461],[181634],[183842],[186073],[188311],[190535],[192718],[194830],[196851],[198771],[200589],[202308],[203938],[205488],[206970],[208399],[209795],[211183],[212589],[214033],[215536],[217116],[218790],[220571],[222460],[224453],[226542],[228715],[230967],[233298],[235712],[238217],[240817],[243512],[246296],[249159],[252085],[255057],[258057],[261063],[264048],[266983],[269839],[272594],[275229],[277728],[280081],[282284],[284338],[286241],[287992]]

Il est important de noter que les processus et variables aléatoires doivent être définies à partir du module 
NdGaussianProcessConstructor, qui contient les classes suivantes : 
###### Classe pour construire un champ stochastique : 
> NdGaussianProcessConstructor.NdGaussianProcessConstructor() 
###### Classe pour construire une variables aléatoire  : 
> NdGaussianProcessConstructor.NormalDistribution() ## Classe openturns.Normal() suchargée
###### Classe pour construire un vecteur de variables aléatoires normales : 
> NdGaussianProcessConstructor.RandomNormalVector() ## Classe openturns.PythonRandomVector surchargée. Cette dernière est utilisée en interne par NdGaussianProcessConstructor.

**Finalement, l'autre particularité du NdGaussianProcessConstructor, est d'utiliser un objet _numpy.memmap_ modifiée, qui enregistre les échantillons de processus gaussiens sous forme de fichier temporaire dans le directoire d'utilisation des codes et les éfface en sortant du code**

L'on définit ensuite les variables de sortie. Il faut connaître l'ordre dans lequel la fonction renvoie ses resultats connaitre leur nom comme leur dimension. 
    

In [16]:
from importlib import reload
reload(spsa)
outputVariables = {'out1' :
                   {
                         'name'     : 'VonMisesStress',
                         'position' : 0,
                         'shape'    : (100,)  
                    },
                   'out2' :
                   {
                        'name'      : 'maxDeflection',
                        'position'  : 1,
                        'shape'     : (1,)
                   }
                  }

###### Ensuite, on crée une instance pour l'analyse de sensibilité.
Il faut donner en entrée une liste contenant les procéssus et les variables aléatoires **dans l'ordre** dans lequel la fonction les recoit. 

*Pour cela il faut aussi connaître l'ordre des variables d'entrée de la fonction et leur nombre.*

**Ces fonctions sont celles qui prennent entrée des champs aléatoires (vecteurs et matrices, de type *list ou numpy*) et variables aléatoire (scalaires (vecteurs 1D pour le multiprocessing)) et non pas les fonctions prennant en entrée les variables aléatoires issues de la décomposition de la Karhunen Loeve, qui elles sont construites en interne dans la classe**

In [17]:
processesDistributions = [process_E, process_D, RV_Fpos, RV_Fnorm]
# We also the need the two functions of the model (one for samples, the other for single evaluations)
# In our case, as our model is defined as a class, we have to first create the model, 
# but it also could just be just two functions taking as an input the fields and RVs
soloFunction   = None
sampleFunction = PBE.batchEval
##
size           = 10000 ## Number of samples for our sobol indicies experiment (kept low here to make things faster)
## It is sufficient just to pass one of the functions 
processSensitivityAnalysis = spsa.StochasticProcessSensitivityAnalysis(processesDistributions, 
                                                                       outputVariables,
                                                                       sampleFunction,
                                                                       soloFunction,
                                                                       size)

Input Variables are (without Karhunen Loeve Decomposition) :
 E_ D_ FP FN 

Output Variables are :
 ['VonMisesStress', 'maxDeflection'] 

Composed distribution built with processes and distributions: E_; D_; FP; FN
Program initialised, ready for sensitivity analysis.         You can now proceed to prepare the Sobol indices experiment



Voici une vue du dictionnaire intermédiaire qui est crée une fois les processus et variables aléatoires d'entrées définies :

On voit que la position des processus dans les arguments de la fonction d'entrée est enregistrée dans le dictionnaire dans la clé *position* et viennent de la manière de laquelle on a mis les VA et Processus dans le vecteur inputVarList. 


###### Ensuite, grace à l'intermédiaire de la classe ot.SobolIndiciesExperiment, on génère les' variables aléatoires d'entrée 

In [18]:
processSensitivityAnalysis.run()

Generation types are:
1 : Random (default)
2 : LHS

3 : LowDiscrepancySequence
4 : SimulatedAnnealingLHS
You choose Random generation
number of samples for sobol experiment = 60000 

input design shape is:  (60000, 27)
DOING EVAL!!!
calling sample function 000
Lifting
Before loop OK, input dim is 4
fieldPositions are [0, 1]
wtf
idxStpPrev= 0 idxStp= 9 k= 0
lifting to field
lifted
Appended
wtf
idxStpPrev= 9 idxStp= 25 k= 1
lifting to field
got exception InvalidDimensionException : Expected coefficients of dimension 16 got 17


In [7]:
processSensitivityAnalysis._wrappedFunction.StochasticProcessList[0][0]

0

###### Ensuite on récupère les sorties associées aux entrées génerées. 
L'idée du postprocessing est d'identifier si dans les sorties il y a des valeurs de type np.nan, et de refaire les experiences manquantes tant qu'il y a des np.nan de présents.

**Ceci crée quelques problèmes : En effet, si le calcul numérique n'a pas pu aboutir avec une certaine réalisation des variables en entrée, est-ce que cela veut dire que cette réalisation est défaillante? Va-t-on l'inclure dans le cas du calcul de la défaillance ou de la sensibilité? Car si on l'inclut pas, il se peut qu'on oublie un nombre conséquent de modes défaillants. Solution : changer le modèle informatique et le rendre robuste à ces erreurs, ou faire une étude précise avec ces réalisation particulières.**


La solution retenue pour génerer les experiences manquantes est d'itérer au dessus de chaque index de la sortie ou se trouvent les nans, de refaire une experience de sobol de taille 1 ( donc qui renverra d+2 valeurs, avec d la dimension de l'entrée), de générer les sorties correspondant aux entrées (recommencer si il s'y trouve un np.nan), et de remplacer les d+2 valeurs du inputDesign d'entrée avec celles que l'on vient de regénérer. 

Le *timing* vient de la classe ***custum_wraps***, qui est pour l'instant assez inutile, mais permet de prendre en main les décorateurs...

Une fois le _output design_ generé, l'on peut faire l'analyse de sensibilité sur le model.
Pour ce faire, l'on utilise la fonction ***self.getSobolIndiciesKLCoefs***.
La fonction ne renvoie pas directement les indices de sobol, mais bien des champs d'objets ***openTURNS.SaltelliSensitivityAlgorithm*** ou ***openTURNS.MartinezSensitivityAlgorithm*** ou ***openTURNS.JansenSensitivityAlgorithm*** ou ***openTURNS.MauntzKucherenkoSensitivityAlgorithm***.

Ceci est bien sûr fait dans l'optique de toujours pouvoir accéder à l'entierté des indices de sobol et indices totaux du modèle.
Ceci veut aussi dire qu'il manque une dimension pour les champs en sortie, celle des variables d'entrée.

Ici on calcule les indices de sobol du premier ordre

In [8]:
processSensitivityAnalysis.getSensitivityAnalysisResults()

You have chosen to use the Saltelli method


TypeError: object of type 'NoneType' has no len()

L'on peut récuperer ces indices du premier ordre dans ***self.processSensitivityAnalysis.sensitivityResults.firstOrderIndices***
Matplotlib permet de facilement visualiser ces indices. Néanmoins, cette visualisation est pour l'instant peu pertinente, puisque l'on visualise chaque variables aléatoire de Karhunen Loeve individuellement, alors qu'il nous faut encore combiner ces dernières pour que l'indice de sobol soit représentatif du champ stochastique consideré.

In [ ]:
processSensitivityAnalysis.SensitivityAnalysisResults['VonMisesStress'].draw()

In [ ]:
processSensitivityAnalysis.SensitivityAnalysisResults['maxDeflection'].draw()